In [1]:
import json
import os

import pandas as pd
import numpy as np

from dotenv import load_dotenv

from core_ds4a_project import cleaning, columns as project_columns, datasets

%load_ext autoreload
%autoreload 1
%aimport core_ds4a_project, core_ds4a_project.cleaning, core_ds4a_project.columns, core_ds4a_project.datasets

pd.set_option("display.max_columns", None)

Environment variables:

In [2]:
load_dotenv('envvars')

ROOT_DATA_PATH = os.environ.get('ROOT_DATA_PATH')
RAW_DATA_PATH = os.environ.get('RAW_DATA_PATH') or f'{ROOT_DATA_PATH}/raw'

Reading raw data:

In [5]:
df = datasets.read_negocio(RAW_DATA_PATH, raw=True)

df.columns

Index(['Homologacion Documento de Identidad', 'Nombre del negocio',
       'Actividad', 'Descripción', 'Referencia Ubicación del negocio',
       'Fecha de inicio del negocio', 'Fecha en el actual lugar de ubicación',
       'Asignado a', 'Método del cálculo de costo de venta', 'Dirección',
       ...
       'Cantidad de Inventario 4',
       'Valor unitario del inventario para el estudio socio-económico 4',
       'Tipo de inventario 5', 'Descripción del Producto 5',
       'Cantidad de Inventario 5',
       'Valor unitario del inventario para el estudio socio-económico 5',
       'Tipo de inventario 6', 'Descripción del Producto 6',
       'Cantidad de Inventario 6',
       'Valor unitario del inventario para el estudio socio-económico 6'],
      dtype='object', length=147)

## Renaming columns

In [7]:
df.columns = cleaning.normalize_columns_name(df.columns)

df.columns.sort_values()  # sorting for presentational purposes only

Index(['ACTIVIDAD', 'ARRENDADOR_DEL_NEGOCIO', 'ASIGNADO_A', 'BARRIO',
       'CANTIDAD_DE_INVENTARIO_1', 'CANTIDAD_DE_INVENTARIO_2',
       'CANTIDAD_DE_INVENTARIO_3', 'CANTIDAD_DE_INVENTARIO_4',
       'CANTIDAD_DE_INVENTARIO_5', 'CANTIDAD_DE_INVENTARIO_6',
       ...
       'VALOR_DE_LA_HIPOTECA_O_LA_PRENDA_1',
       'VALOR_DE_LA_HIPOTECA_O_LA_PRENDA_2',
       'VALOR_DE_LA_HIPOTECA_O_LA_PRENDA_3',
       'VALOR_UNITARIO_DEL_INVENTARIO_PARA_EL_ESTUDIO_SOCIO-ECONOMICO_1',
       'VALOR_UNITARIO_DEL_INVENTARIO_PARA_EL_ESTUDIO_SOCIO-ECONOMICO_2',
       'VALOR_UNITARIO_DEL_INVENTARIO_PARA_EL_ESTUDIO_SOCIO-ECONOMICO_3',
       'VALOR_UNITARIO_DEL_INVENTARIO_PARA_EL_ESTUDIO_SOCIO-ECONOMICO_4',
       'VALOR_UNITARIO_DEL_INVENTARIO_PARA_EL_ESTUDIO_SOCIO-ECONOMICO_5',
       'VALOR_UNITARIO_DEL_INVENTARIO_PARA_EL_ESTUDIO_SOCIO-ECONOMICO_6',
       'ZONA'],
      dtype='object', length=147)

## Dropping inconsistent information

There is multiple inconsistent information for some CLIENTE records:

In [10]:
ind = df['CLIENTE'].duplicated(keep=False)

{
    "count_inconsistent_records": ind.sum()
}


{'count_inconsistent_records': 2363}

For instance:

In [12]:
df.loc[ind, ['CLIENTE', 'ACTIVIDAD', 'NOMBRE_DEL_NEGOCIO']].query('CLIENTE == "FA10007"')

,CLIENTE,ACTIVIDAD,NOMBRE_DEL_NEGOCIO
9582,FA10007,Servicios |S,SALON DE BELLEZA Y ARRENDAMIENTOS tienda
21118,FA10007,Comercio |C,DISTRIBUIDORA DE POLLO


Dropping inconsistent records:

In [13]:
df = df[~ind]